In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics -q

In [ ]:
!pip install tensorflow -q

In [ ]:
!pip install torch -q

In [ ]:
!nvidia-smi

Wed Feb  7 14:31:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0              29W /  70W |    719MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.10 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.5/78.2 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
model = YOLO('yolov8m.pt')
results = model.track(source='/content/drive/MyDrive/unitedvsvilla720p.mp4', conf=0.25, save=True, tracker='bytetrack.yaml', classes=0)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/160) /content/drive/MyDrive/unitedvsvilla720p.mp4: 384x640 12 persons, 26.6ms
video 1/1 (2/160) /content/drive/MyDrive/unitedvsvilla720p.mp4: 384x640 12 persons, 25.8ms
video 1/1 (3/160) /content/drive/MyDrive/unitedvsvilla720p.mp4: 384x640 12 persons, 25.5ms
video 1/1 (4/160) /content/drive/MyDrive/unitedvsvilla720p.mp4: 384x640 12 persons, 25.6ms
video 1/1 (5/160) /content/drive/MyDrive/unitedvsvilla720p.mp4: 384x640 12 persons, 25.5ms
v

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from glob import glob
from random import sample
from PIL import Image, ImageFont, ImageDraw, ImageEnhance, ImageFilter
from pathlib import Path
import os
import shutil
import numpy as np
import cv2
import random
import torch
from tensorflow.keras.preprocessing.image import array_to_img

In [ ]:
last_frame_index = len(results) - 1
last_frame_boxes = results[last_frame_index].boxes.xyxy.tolist()
print(last_frame_boxes)

# Provided tap coordinates of the user on the last frame
tap_coord = (450, 353)
# ID of the bounding box that the user tapped on. We only need to save crops of this bounding box.
required_track_id = None

# Iterate through each box in the last_frame_boxes
for index, box in enumerate(last_frame_boxes):
    x_min, y_min, x_max, y_max = box  # Unpack box coordinates
    if x_min <= tap_coord[0] <= x_max and y_min <= tap_coord[1] <= y_max:
        # The target point is within this box
        print(f"Box at index {index} is within the coordinates")
        required_track_id = results[last_frame_index].boxes[index].id.tolist()[0]
        print(f"Required tracking ID is: {required_track_id}")
        break

[[584.1489868164062, 327.4786071777344, 614.4602661132812, 397.74615478515625], [786.3281860351562, 287.3955993652344, 818.7329711914062, 356.2096252441406], [856.7017822265625, 215.59715270996094, 886.5623779296875, 269.9825134277344], [1123.8258056640625, 329.0048828125, 1164.3280029296875, 401.58740234375], [867.2447509765625, 472.5174255371094, 910.713134765625, 566.4974975585938], [753.9646606445312, 251.99551391601562, 779.2638549804688, 304.7622985839844], [821.1785278320312, 234.31822204589844, 854.2954711914062, 290.9352722167969], [439.3404541015625, 332.616943359375, 474.4425048828125, 399.2972106933594], [547.2337036132812, 302.69287109375, 573.4224853515625, 361.7489929199219]]
Box at index 7 is within the coordinates
Required tracking ID is: 3.0


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/yolov5ID_640_100epoch.pt')
model.load_state_dict(torch.load('/content/drive/MyDrive/yolov5ID_640_100epoch.pt')['model'].state_dict(), strict=False)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-2-7 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7279825 parameters, 0 gradients, 16.6 GFLOPs
Adding AutoShape... 


_IncompatibleKeys(missing_keys=['model.model.model.0.conv.weight', 'model.model.model.0.conv.bias', 'model.model.model.1.conv.weight', 'model.model.model.1.conv.bias', 'model.model.model.2.cv1.conv.weight', 'model.model.model.2.cv1.conv.bias', 'model.model.model.2.cv2.conv.weight', 'model.model.model.2.cv2.conv.bias', 'model.model.model.2.cv3.conv.weight', 'model.model.model.2.cv3.conv.bias', 'model.model.model.2.m.0.cv1.conv.weight', 'model.model.model.2.m.0.cv1.conv.bias', 'model.model.model.2.m.0.cv2.conv.weight', 'model.model.model.2.m.0.cv2.conv.bias', 'model.model.model.3.conv.weight', 'model.model.model.3.conv.bias', 'model.model.model.4.cv1.conv.weight', 'model.model.model.4.cv1.conv.bias', 'model.model.model.4.cv2.conv.weight', 'model.model.model.4.cv2.conv.bias', 'model.model.model.4.cv3.conv.weight', 'model.model.model.4.cv3.conv.bias', 'model.model.model.4.m.0.cv1.conv.weight', 'model.model.model.4.m.0.cv1.conv.bias', 'model.model.model.4.m.0.cv2.conv.weight', 'model.model.

In [ ]:
# Specify the path to the folder where you want to save the cropped images
output_folder = '/content/cropped_images/'

# Create the folder if it doesn't exist
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

In [ ]:
def yolov5(img):
    im = array_to_img(img)
    output = model(im)
    results = output.pandas().xyxy[0].to_dict(orient="records")
    for result in results:
        con = result['confidence']
        cs = result['class']
        return cs

In [ ]:
image_paths = []

for result_index, result in enumerate(results):
    # Iterate through all boxes in the current frame
    for box_index, box in enumerate(result.boxes):
        current_track_id = box.id.tolist()[0]

        # Check if the box's ID matches the required_track_id
        if current_track_id == required_track_id:

            # Get the box coordinates
            x_min, y_min, x_max, y_max = box.xyxy.tolist()[0]
            orig_img_bgr = result.orig_img

            # Convert BGR to RGB
            orig_img_rgb = cv2.cvtColor(orig_img_bgr, cv2.COLOR_BGR2RGB)

            # Create PIL Image from RGB array
            orig_img = Image.fromarray(orig_img_rgb)

            # Crop the image using adjusted box coordinates
            cropped_img = orig_img.crop((x_min, y_min, x_max, y_max))

            # Increase the size of the cropped image
            enlarged_size = (cropped_img.width * 2, cropped_img.height * 2)
            cropped_img = cropped_img.resize(enlarged_size, Image.BICUBIC)

            # Sharpen the image
            cropped_img_sharpened = cropped_img.filter(ImageFilter.SHARPEN)
            cropped_img_sharpened = cropped_img_sharpened.filter(ImageFilter.SHARPEN)

            # Save the sharpened and enlarged image to the specified folder
            image_filename = f'cropped_image_{result_index}_box_{box_index}_sharpened.png'
            image_path = os.path.join(output_folder, image_filename)
            cropped_img_sharpened.save(image_path)

            image_paths.append(image_path)


In [ ]:
num_dict = {}

for path in image_paths:
  img = cv2.imread(path)
  num = yolov5(img)
  if num is not None:
    if num not in num_dict:
      num_dict[num] = 1
    else:
      num_dict[num] += 1
  #print(f"Detected Jersey Number: {num}")

print(num_dict)
print("Jersey number is: ", max(num_dict, key=num_dict.get))

{10: 4, 13: 2, 33: 1, 38: 1}
Jersey number is:  10
